In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()


# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml inference_config

experiment:
    exp_root: '?'
    seed: 42

calibration:
    num_bins: 15
    neighborhood_width: 3
    square_diff: False 

data:
    split: cal 
    preload: '?' 

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

log:
    root: '?'
    log_interval: 10 
    show_examples: '?' 
    log_image_stats: '?' 
    log_pixel_stats: '?' 
    track_classwise_ps: False

<IPython.core.display.Javascript object>

In [3]:
%%yaml model_cfg

model:
    pretrained_exp_root : '?' 
    calibrated: '?'
    checkpoint: '?' 
    ensemble: '?' 
    ensemble_cfg: '?'
    ensemble_w_metric: '?'
    pretrained_select_metric: "val-dice_score"

<IPython.core.display.Javascript object>

In [4]:
%%yaml cal_metrics_cfg 

image_cal_metrics:
    # ECE Metrics
    # - Image_ECE:
    #     _fn: ese.experiment.metrics.ece.image_ece_loss
    # - Image_Edge-ECE:
    #     _fn: ese.experiment.metrics.ece.image_edge_ece_loss
    - Image_CW-ECE:
        _fn: ese.experiment.metrics.ece.image_cw_ece_loss
    # - Image_ECW_ECE:
    #     _fn: ese.experiment.metrics.ece.image_ecw_ece_loss
    # # ELM Metrics
    # - Image_ELM:
    #     _fn: ese.experiment.metrics.elm.image_elm_loss
    # - Image_Edge-ELM:
    #     _fn: ese.experiment.metrics.elm.image_edge_elm_loss
    # - Image_CW-ELM:
    #     _fn: ese.experiment.metrics.elm.image_cw_elm_loss
    # - Image_ECW_ELM:
    #     _fn: ese.experiment.metrics.elm.image_ecw_elm_loss
    # # Foreground ECE Metrics
    # - Image_Foreground-ECE:
    #     _fn: ese.experiment.metrics.ece.image_ece_loss
    #     ignore_index: 0
    # - Image_Foreground-Edge-ECE:
    #     _fn: ese.experiment.metrics.ece.image_edge_ece_loss
    #     ignore_index: 0
    # - Image_Foreground-CW-ECE:
    #     _fn: ese.experiment.metrics.ece.image_cw_ece_loss
    #     ignore_index: 0
    # - Image_Foreground-ECW-ECE:
    #     _fn: ese.experiment.metrics.ece.image_ecw_ece_loss
    #     ignore_index: 0
    # # Foreground ELM Metrics
    # - Image_Foreground-ELM:
    #     _fn: ese.experiment.metrics.elm.image_elm_loss
    #     ignore_index: 0
    # - Image_Foreground-Edge-ELM:
    #     _fn: ese.experiment.metrics.elm.image_edge_elm_loss
    #     ignore_index: 0
    # - Image_Foreground-CW-ELM:
    #     _fn: ese.experiment.metrics.elm.image_cw_elm_loss
    #     ignore_index: 0
    # - Image_Foreground-ECW-ELM:
    #     _fn: ese.experiment.metrics.elm.image_ecw_elm_loss
    #     ignore_index: 0

# Define global metrics when you want to verify that these are the same as the image
# based metrics, per image.
global_cal_metrics:
    # ECE Metrics
    # - ECE:
    #     _fn: ese.experiment.metrics.ece.ece_loss
    # - Edge-ECE:
    #     _fn: ese.experiment.metrics.ece.edge_ece_loss
    - CW-ECE:
        _fn: ese.experiment.metrics.ece.cw_ece_loss
    # - ECW_ECE:
    #     _fn: ese.experiment.metrics.ece.ecw_ece_loss
    # # ELM Metrics
    # - elm:
    #     _fn: ese.experiment.metrics.elm.elm_loss
    # - edge-elm:
    #     _fn: ese.experiment.metrics.elm.edge_elm_loss
    # - cw-elm:
    #     _fn: ese.experiment.metrics.elm.cw_elm_loss
    # - ecw_elm:
    #     _fn: ese.experiment.metrics.elm.ecw_elm_loss
    # # Foreground ECE Metrics
    # - Foreground-ECE:
    #     _fn: ese.experiment.metrics.ece.ece_loss
    #     ignore_index: 0
    # - Foreground-Edge-ECE:
    #     _fn: ese.experiment.metrics.ece.edge_ece_loss
    #     ignore_index: 0
    # - Foreground-CW-ECE:
    #     _fn: ese.experiment.metrics.ece.cw_ece_loss
    #     ignore_index: 0
    # - Foreground-ECW-ECE:
    #     _fn: ese.experiment.metrics.ece.ecw_ece_loss
    #     ignore_index: 0
    # # Foreground ELM Metrics
    # - Foreground-ELM:
    #     _fn: ese.experiment.metrics.elm.elm_loss
    #     ignore_index: 0
    # - Foreground-Edge-ELM:
    #     _fn: ese.experiment.metrics.elm.edge_elm_loss
    #     ignore_index: 0
    # - Foreground-CW-ELM:
    #     _fn: ese.experiment.metrics.elm.cw_elm_loss
    #     ignore_index: 0
    # - Foreground-ECW-ELM:
    #     _fn: ese.experiment.metrics.elm.ecw_elm_loss
    #     ignore_index: 0

<IPython.core.display.Javascript object>

In [5]:
from typing import List, Optional
from ese.scripts.utils import gather_exp_paths


def get_ese_inference_configs(
    group_dict: dict,
    calibrators_list: List[str], 
    do_ensemble: bool, 
    log_image_stats: bool = True,
    log_pixel_stats: bool = True,
    show_examples: bool = False,
    ensemble_upper_bound: bool = False,
    ens_cfg_options: Optional[List[tuple]] = None,
    ens_w_metric_list: Optional[List[str]] = ['None'],
):
    if not do_ensemble:
        assert ens_w_metric_list == ['None'], "No weighting can be provided if no ensemble."
        assert ens_cfg_options == None, "No ensemble config can be provided if no ensemble."
        assert not ensemble_upper_bound, "No ensemble upper bound can be provided if no ensemble."

    if ens_cfg_options is None:
        ens_cfg_options = [None]

    calibrator_option_list = []
    for calibrator in calibrators_list:
        for ensemble_w_metric in ens_w_metric_list:
            model_calibrated = calibrator != "Uncalibrated"
            # Define the paths for the uncailbrated networks.
            ##################################################
            if calibrator == "Uncalibrated":
                ensemble_root = f"/storage/vbutoi/scratch/ESE/training/{group_dict['base_models_group']}"
                checkpoint = "max-val-dice_score" 
            # Define the paths for the calibrated networks.
            ##################################################
            else:
                ensemble_root = f"/storage/vbutoi/scratch/ESE/calibration/{group_dict['calibrated_models_group']}/Individual_{calibrator}"
                checkpoint = "min-val-ece_loss"

            # Set a few things that will be consistent for all runs.
            ##################################################
            exp_root = root / "inference" / group_dict['exp_group']

            default_config_options = {
                'experiment.exp_root': [str(exp_root)],
                'model.checkpoint': [checkpoint],
                'model.calibrated': [model_calibrated],
                'data.preload': [group_dict['preload']],
                'log.show_examples': [show_examples],
                'log.log_image_stats': [log_image_stats],
                'log.log_pixel_stats': [log_pixel_stats],
            }
            # Make presets for the different runnning configurations.
            ##################################################
            # If you want to run inference on ensembles, use this.
            if do_ensemble:
                if ensemble_upper_bound:
                    log_root = str(exp_root / f"ensemble_upper_bounds")
                else:
                    log_root = str(exp_root / f"{group_dict['dataset']}_Ensemble_{calibrator}")
                advanced_args = {
                    'log.root': [log_root],
                    'model.pretrained_exp_root': [ensemble_root],
                    'model.ensemble': [True],
                    'model.ensemble_w_metric': [ensemble_w_metric],
                    'model.ensemble_cfg': ens_cfg_options 
                }
            # If you want to run inference on individual networks, use this.
            else:
                advanced_args = {
                    'log.root': [str(exp_root / f"{group_dict['dataset']}_Individual_{calibrator}")],
                    'model.pretrained_exp_root': gather_exp_paths(ensemble_root), # Note this is a list of train exp paths.
                    'model.ensemble': [False],
                    'model.ensemble_cfg': ens_cfg_options,
                    'model.ensemble_w_metric': [None]
                }
            # Combine the default and advanced arguments.
            default_config_options.update(advanced_args)
            # Append these to the list of configs and roots.
            calibrator_option_list.append(default_config_options)
    # Return the list of different configs.
    return calibrator_option_list

In [6]:
# exp_name = '01_29_24_WMH_Base'
# group_dict = {
#     "dataset": "WMH",
#     "exp_group": exp_name,
#     "base_models_group": "01_08_24_WMH_Ensemble",
#     "calibrated_models_group": "01_18_24_WMH_All_Calibrators",
#     "preload": False
# }

exp_name = '01_29_24_WMH_Foreground' 
group_dict = {
    "dataset": "WMH",
    "exp_group": exp_name,
    "base_models_group": "01_08_24_WMH_Ensemble",
    "calibrated_models_group": "01_19_24_WMH_Foreground_Calibrators",
    "preload": False
}

# exp_name = '01_31_24_CityScapes_AllCalibrators' 
# group_dict = {
#     "dataset": "CityScapes",
#     "exp_group": exp_name,
#     "base_models_group": "01_25_24_CityScapes_Dice",
#     "calibrated_models_group": "01_26_24_CityScapes_Ensemble",
#     "preload": False
# }

# Get the configs for the different runs.
dataset_options = get_ese_inference_configs(
    group_dict=group_dict,
    do_ensemble=True, 
    calibrators_list=[
        # 'Uncalibrated',
        # 'Vanilla',
        # 'TempScaling',
        # 'VectorScaling',
        'DirichletScaling',
        # 'LTS',
        # 'NectarScaling'
    ], 
    ens_cfg_options=[
        ('mean', 'logits'), 
        ('mean', 'probs'), 
        ('product', 'probs')
    ],
    ens_w_metric_list=[
        'None',
        # 'val-loss',
        # 'val-dice_score',
        # 'val-ece_loss',
        # 'val-elm_loss',
    ],
)

In [7]:
# # Get the configs for computing the upepr bounds
#
# dataset_options = get_ese_inference_configs(
#     group_dict=group_dict,
#     do_ensemble=True, 
#     ensemble_upper_bound=True,
#     calibrators_list=[
#         'Uncalibrated',
#     ], 
#     ens_cfg_options=[
#         ('upperbound', 'probs')
#     ]
# )

In [8]:
from ese.scripts.utils import get_option_product
from ionpy.util import Config
import yaml

# Load the inference cfg from local.
##################################################
with open(code_root / "ese" / "experiment" / "configs" / "inference" / f"{group_dict['dataset']}.yaml", 'r') as file:
    dataset_inference_cfg = yaml.safe_load(file)
##################################################
base_cfg = Config(inference_config).update([model_cfg, cal_metrics_cfg, dataset_inference_cfg])

# Get the configs
cfgs = get_option_product(exp_name, dataset_options, base_cfg)

In [9]:
len(cfgs)

3

## Running Jobs

In [10]:
from ese.experiment.experiment import run_ese_exp, submit_ese_exps
from ese.experiment.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [11]:
###### Run individual jobs
run_ese_exp(
    config=cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    gpu='0',
) 

Set seed: 42


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


  0%|          | 0/6 [00:00<?, ?it/s]

Set seed: 43
Set seed: 43


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Set seed: 40
Set seed: 40
Set seed: 41
Set seed: 41
Set seed: 42
Set seed: 42


  0%|          | 0/4 [00:00<?, ?it/s]

Set seed: 42
Running:

calibration:
  neighborhood_width: 3
  num_bins: 15
  square_diff: false
data:
  preload: false
  slicing: dense_full
  split: cal
dataloader:
  batch_size: 1
  num_workers: 0
  pin_memory: true
dataset:
  _class: ese.experiment.datasets.WMH
  annotator: observer_o12
  augmentations: null
  axis: 0
  num_slices: 1
  preload: false
  return_data_id: true
  slicing: dense_full
  split: cal
  task: Amsterdam
  version: 0.2
experiment:
  exp_root: /storage/vbutoi/scratch/ESE/inference/01_29_24_WMH_Foreground
  pretrained_seed: 42
  seed: 42
global_cal_metrics:
- CW-ECE:
    _fn: ese.experiment.metrics.ece.cw_ece_loss
image_cal_metrics:
- Image_CW-ECE:
    _fn: ese.experiment.metrics.ece.image_cw_ece_loss
log:
  log_image_stats: true
  log_interval: 10
  log_pixel_stats: true
  root: /storage/vbutoi/scratch/ESE/inference/01_29_24_WMH_Foreground/debug
  show_examples: false
  track_classwise_ps: false
  track_label_amounts: true
  wandb_string: exp_name:01_29_24_WMH_Fo

ValueError: WARNING on data id 103, slice 0: CALIBRATION METRIC 'Image_CW-ECE' DOES NOT MATCH FOR IMAGE AND PIXEL LEVELS. Pixel level calibration score (0.5) does not match image level score (1.0).

In [ ]:
# ####### Run Batch Jobs
# submit_ese_exps(
#     config_list=cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
# )

## Here we run upper-bounds.

In [ ]:
from ese.experiment.analysis.ensemble_upperbound import get_ensemble_ub

In [ ]:
# ###### Run individual jobs
# run_ese_exp(
#     config=cfgs[0], 
#     job_func=get_ensemble_ub,
#     run_name='debug',
#     gpu='0',
# ) 

In [ ]:
# ####### Run Batch Jobs
# submit_ese_exps(
#     config_list=cfgs,
#     job_func=get_ensemble_ub,
#     available_gpus=['1']
# )